<a href="https://colab.research.google.com/github/Baludatascience/Sentiment-Analysis-using-LSTM/blob/master/LSTM_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Import necessary packages/libraries 

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

KeyboardInterrupt: 

In [2]:
data = pd.read_csv(r"C:\Users\HP\Downloads\archive (6)\IMDB Dataset.csv",encoding='latin-1')

 ## reading the data from folder

#Considering only the necessary columns#
data = data[['review', 'sentiment']]
data.shape     

(50000, 2)

In [ ]:
# Performing data preprocessing##


data ['review']  = data['review'].apply(lambda x: x.lower())                            #Converts all text in the review column to lowercase.
data ['review'] = data['review'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', ' ', x)))    #Cleans the text data by removing special characters and punctuation.


print( data[data['sentiment'] == 'positive'].size)
print( data[data['sentiment'] == 'negative'].size)

# for idx,row in data.iterrows():
#   row[0] =row[0].replace('rt', ' ') ##

max_fatures = 20000
tokenizer = Tokenizer(num_words = max_fatures, split= ' ')          #Initializes a Tokenizer object for processing text data.
tokenizer.fit_on_texts(data['review'].values)                       # Updates the tokenizer's internal vocabulary based on the review column.
                                                                    #data['review'].values: Extracts the review texts as a NumPy array.
                                                                    #fit_on_texts: Learns the word-to-index mapping from the text data.

X = tokenizer.texts_to_sequences(data['review'].values)             #Purpose: Converts the review texts into sequences of integers based on the tokenizer's word-to-index mapping.
                                                                    #Usage:
                                                                    #Each word in the review text is replaced by its corresponding integer index.
                                                                    #Produces a list of lists, where each inner list represents the integer sequence for a review.


X = pad_sequences(X)                                                #Purpose: Pads the sequences to ensure uniform length.
                                                                    # Usage:
                                                                    # pad_sequences: Adds padding (usually zeros) to shorter sequences or truncates longer sequences to a fixed length.
                                                                    # Necessary because deep learning models require inputs of consistent dimensions.



<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\HP\AppData\Local\Temp\ipykernel_5368\845890803.py:5: SyntaxWarning: invalid escape sequence '\s'
  data ['review'] = data['review'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', ' ', x)))


50000
50000


C:\Users\HP\AppData\Local\Temp\ipykernel_5368\845890803.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] =row[0].replace('rt', ' ') ##
C:\Users\HP\AppData\Local\Temp\ipykernel_5368\845890803.py:11: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] =row[0].replace('rt', ' ') ##


In [5]:
#Creating layers using LSTM
embed_dim = 128    
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [6]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train,Y_test = train_test_split(X,Y, test_size = 0.33, random_state= 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(33500, 2367) (33500, 2)
(16500, 2367) (16500, 2)


In [8]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)

1047/1047 - 8463s - 8s/step - accuracy: 0.8097 - loss: 0.4291


In [7]:
!output_directory new/

/bin/bash: output_directory: command not found


In [9]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

469/469 - 300s - 640ms/step - accuracy: 0.8487 - loss: 0.3687
score: 0.37
acc: 0.85


In [10]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1


print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 1s - 751ms/step
1/1 - 0s - 363ms/step
1/1 - 0s - 358ms/step
1/1 - 0s - 367ms/step
1/1 - 0s - 360ms/step
1/1 - 0s - 361ms/step
1/1 - 0s - 362ms/step
1/1 - 0s - 360ms/step
1/1 - 0s - 355ms/step
1/1 - 0s - 365ms/step
1/1 - 0s - 390ms/step
1/1 - 0s - 388ms/step
1/1 - 0s - 376ms/step
1/1 - 0s - 348ms/step
1/1 - 0s - 348ms/step
1/1 - 0s - 438ms/step
1/1 - 0s - 392ms/step
1/1 - 0s - 344ms/step
1/1 - 0s - 355ms/step
1/1 - 0s - 359ms/step
1/1 - 0s - 358ms/step
1/1 - 0s - 392ms/step
1/1 - 0s - 361ms/step
1/1 - 0s - 386ms/step
1/1 - 0s - 459ms/step
1/1 - 1s - 509ms/step
1/1 - 0s - 412ms/step
1/1 - 0s - 364ms/step
1/1 - 1s - 1s/step
1/1 - 0s - 450ms/step
1/1 - 0s - 481ms/step
1/1 - 0s - 428ms/step
1/1 - 0s - 424ms/step
1/1 - 0s - 416ms/step
1/1 - 0s - 460ms/step
1/1 - 0s - 442ms/step
1/1 - 0s - 484ms/step
1/1 - 1s - 606ms/step
1/1 - 0s - 402ms/step
1/1 - 0s - 265ms/step
1/1 - 0s - 430ms/step
1/1 - 0s - 412ms/step
1/1 - 0s - 431ms/step
1/1 - 0s - 430ms/step
1/1 - 0s - 409ms/step
1/1 - 0s - 41

In [17]:
##
twt  = ['i am feeling very good,not at all bad']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=30, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt, batch_size=1, verbose=2)[0]
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  10 236 555  54  50  24  32  31  75]]
1/1 - 0s - 40ms/step
1/1 - 0s - 48ms/step
negative
